In [1]:
from datapoint import DataPoint
import numpy as np

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource
output_notebook()

Loading BokehJS ...

In [2]:
def parse_in_data(file_path):
  """
    Args:
    file_path 

          R 8.46642 0.0287602 -3.04035  1477010443399637  8.6 0.25  -3.00029  0
          (R) (rho) (phi) (drho) (timestamp) (real x) (real y) (real vx) (real vy)

          L 8.44818 0.251553  1477010443449633  8.45  0.25  -3.00027  0
          (R) (x) (y) (timestamp) (real x) (real y) (real vx) (real vy)

    Returns:
      all_sensor_data, all_ground_truths
      - two lists of DataPoint() instances 

  """

  all_sensor_data = []
  all_ground_truths = []  

  with open(file_path) as f:
      
    for line in f:
      data = line.split() 
      
      if data[0]  == 'L':
        
        sensor_data = DataPoint({ 
          'timestamp': int(data[3]),
          'name': 'lidar',
          'x': float(data[1]), 
          'y': float(data[2])
        })
        
        g = {'timestamp': int(data[3]),
             'name': 'state',
             'x': float(data[4]),
             'y': float(data[5]),
             'vx': float(data[6]),
             'vy': float(data[7])
        }
          
        ground_truth = DataPoint(g)
                
      elif data[0] == 'R':
          
        sensor_data = DataPoint({ 
          'timestamp': int(data[4]),
          'name': 'radar',
          'rho': float(data[1]), 
          'phi': float(data[2]),
          'drho': float(data[3])
        })
       
        g = {'timestamp': int(data[4]),
             'name': 'state',
             'x': float(data[5]),
             'y': float(data[6]),
             'vx': float(data[7]),
             'vy': float(data[8])
        }
        ground_truth = DataPoint(g)  

      all_sensor_data.append(sensor_data)
      all_ground_truths.append(ground_truth)

  return all_sensor_data, all_ground_truths

In [3]:
def parse_prediction_data(file_path):
  
  all_predictions = []
    
  with open(file_path) as f:
    
    for line in f:
      data = line.split()
      
      p = {'timestamp': None,
           'name': 'state',
           'x': float(data[0]),
           'y': float(data[1]),
           'vx': float(data[2]),
           'vy': float(data[3])
        }
    
      all_predictions.append(DataPoint(p))
        
  return all_predictions

In [4]:
all_sensor_data, all_ground_truths = parse_in_data("data/data-1.txt")
all_state_estimations = parse_prediction_data("data/out-1-B.txt")

In [5]:
lidar_xs, lidar_ys = [], []
radar_xs, radar_ys, radar_angles = [], [], []
truth_xs, truth_ys, truth_angles = [], [], []
state_xs, state_ys, state_angles = [], [], []
    
for s, t, p in zip(all_sensor_data, all_ground_truths, all_state_estimations):
    
  if s.get_name() == "lidar":
    x, y = s.get_raw()
    lidar_xs.append(x)
    lidar_ys.append(y)
  else:
    x, y, vx, vy = s.get()
    angle = np.arctan2(vy, vx)
    radar_xs.append(x)
    radar_ys.append(y)
    radar_angles.append(angle)

  x, y, vx, vy = t.get()
  t_angle =  np.arctan2(vy, vy)
  truth_xs.append(x)
  truth_ys.append(y)
  truth_angles.append(t_angle)

  x, y, vx, vy = p.get()
  p_angle =  np.arctan2(vy, vx)
  state_xs.append(x)
  state_ys.append(y)
  state_angles.append(p_angle)

In [6]:
radar_source = ColumnDataSource(data = {
    'x' : radar_xs,
    'y' : radar_ys,
    'angle': radar_angles, 
})

truth_source = ColumnDataSource(data = {
    'x' : truth_xs,
    'y' : truth_ys,
    'angle': truth_angles, 
})

state_source = ColumnDataSource(data = {
    'x' : state_xs,
    'y' : state_ys,
    'angle': truth_angles, 
})

lidar_source = ColumnDataSource(data = {
    'x' : lidar_xs,
    'y' : lidar_ys,
})

In [7]:
hover1 = HoverTool( tooltips = [
            ("index", "$index"),
            ("x , y", "$x, $y"),
            ("angle radians", "@angle")])

p = figure(plot_width = 1000, plot_height = 700, tools = [hover1])

p.square(
  'x', 'y', size = 5, 
  fill_color = "orange",
  line_color = "orange",
  fill_alpha = 1,
  angle = 'angle', 
  line_width = 1, 
  legend = "radar measurements", 
  source = radar_source)

p.circle(
  'x', 'y', size = 15, 
  fill_color = "green",
  line_color = "white",
  fill_alpha = 0.2,
  line_width = 1, 
  legend = "lidar measurements", 
  source = lidar_source)

p.triangle(
  'x', 'y', size = 7, 
  fill_color = "violet",
  line_color = "violet",
  fill_alpha = 1,
  angle = 'angle', 
  line_width = 0.5, 
  legend = "ground truth",
  source = truth_source)


p.triangle(
  'x', 'y', size = 2, 
  fill_color = "black",
  line_color = "black",
  fill_alpha = 1,
  angle = 'angle', 
  line_width = 1, 
  legend = "state predictions", 
  source = state_source)

p.legend.location = "bottom_right"
show(p)

In [8]:
hover2 = HoverTool( tooltips = [
            ("index", "$index"),
            ("x , y", "$x, $y"),
            ("angle radians", "@angle")])

p = figure(plot_width = 1000, plot_height = 700, tools = [hover2])

p.square(
  'x', 'y', size = 5, 
  fill_color = "violet",
  line_color = "violet",
  fill_alpha = 1,
  angle = 'angle', 
  line_width = 1, 
  legend = "radar measurements", 
  source = radar_source)

p.circle(
  'x', 'y', size = 10, 
  fill_color = "green",
  line_color = "white",
  fill_alpha = 0.4,
  line_width = 1, 
  legend = "lidar measurements", 
  source = lidar_source)

p.line(state_xs, state_ys, line_width = 2, color='orange', legend = "state predictions")
p.line(truth_xs, truth_ys, line_dash = "4 4", line_width = 1, color='navy', legend = "ground truth")

p.legend.location = "bottom_right"
show(p)

In [9]:
all_sensor_data, all_ground_truths = parse_in_data("data/data-2.txt")
all_state_estimations = parse_prediction_data("data/out-2-B.txt")

In [10]:
lidar_xs, lidar_ys = [], []
radar_xs, radar_ys, radar_angles = [], [], []
truth_xs, truth_ys, truth_angles = [], [], []
state_xs, state_ys, state_angles = [], [], []
    
for s, t, p in zip(all_sensor_data, all_ground_truths, all_state_estimations):
    
  if s.get_name() == "lidar":
    x, y = s.get_raw()
    lidar_xs.append(x)
    lidar_ys.append(y)
  else:
    x, y, vx, vy = s.get()
    angle = np.arctan2(vy, vx)
    radar_xs.append(x)
    radar_ys.append(y)
    radar_angles.append(angle)

  x, y, vx, vy = t.get()
  t_angle =  np.arctan2(vy, vy)
  truth_xs.append(x)
  truth_ys.append(y)
  truth_angles.append(t_angle)

  x, y, vx, vy = p.get()
  p_angle =  np.arctan2(vy, vx)
  state_xs.append(x)
  state_ys.append(y)
  state_angles.append(p_angle)

radar_source = ColumnDataSource(data = {
    'x' : radar_xs,
    'y' : radar_ys,
    'angle': radar_angles, 
})

truth_source = ColumnDataSource(data = {
    'x' : truth_xs,
    'y' : truth_ys,
    'angle': truth_angles, 
})

state_source = ColumnDataSource(data = {
    'x' : state_xs,
    'y' : state_ys,
    'angle': truth_angles, 
})

lidar_source = ColumnDataSource(data = {
    'x' : lidar_xs,
    'y' : lidar_ys,
})

In [11]:
hover3 = HoverTool( tooltips = [
            ("index", "$index"),
            ("x , y", "$x, $y"),
            ("angle radians", "@angle")])

p = figure(plot_width = 1000, plot_height = 700, tools = [hover3])

p.square(
  'x', 'y', size = 5, 
  fill_color = "orange",
  line_color = "orange",
  fill_alpha = 1,
  angle = 'angle', 
  line_width = 1, 
  legend = "radar measurements", 
  source = radar_source)

p.circle(
  'x', 'y', size = 15, 
  fill_color = "green",
  line_color = "white",
  fill_alpha = 0.2,
  line_width = 1, 
  legend = "lidar measurements", 
  source = lidar_source)

p.triangle(
  'x', 'y', size = 7, 
  fill_color = "violet",
  line_color = "violet",
  fill_alpha = 1,
  angle = 'angle', 
  line_width = 0.5, 
  legend = "ground truth",
  source = truth_source)


p.triangle(
  'x', 'y', size = 2, 
  fill_color = "black",
  line_color = "black",
  fill_alpha = 1,
  angle = 'angle', 
  line_width = 1, 
  legend = "state predictions", 
  source = state_source)

p.legend.location = "bottom_right"
show(p)

In [12]:
hover4 = HoverTool( tooltips = [
            ("index", "$index"),
            ("x , y", "$x, $y"),
            ("angle radians", "@angle")])

p = figure(plot_width = 1000, plot_height = 700, tools = [hover4])

p.square(
  'x', 'y', size = 5, 
  fill_color = "violet",
  line_color = "violet",
  fill_alpha = 1,
  angle = 'angle', 
  line_width = 1, 
  legend = "radar measurements", 
  source = radar_source)

p.circle(
  'x', 'y', size = 10, 
  fill_color = "green",
  line_color = "white",
  fill_alpha = 0.4,
  line_width = 1, 
  legend = "lidar measurements", 
  source = lidar_source)

p.line(state_xs, state_ys, line_width = 2, color='orange', legend = "state predictions")
p.line(truth_xs, truth_ys, line_dash = "4 4", line_width = 1, color='navy', legend = "ground truth")

p.legend.location = "bottom_right"
show(p)